# This notebook is used to read the CMIP6 data, calculating the seasonal (annual) means and combine all the data into one dataset

In [1]:
import xarray as xr
import pandas as pd
from scipy import stats
import numpy as np
from cartopy import crs as ccrs, feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
from metpy.units import units
import matplotlib.colors as colors
import metpy.calc as mpcalc
import glob,os
# %run functions/Yearly_mean.ipynb
# %run functions/Seasonal_means.ipynb
# %run functions/Transfer_longitude.ipynb

## Function for calculating the seasonal means

In [1]:
def season_mean(ds, calendar="standard"):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )
    # Calculate the weighted average
#     ds_weighted=(ds * weights).groupby("time.season").sum(dim="time")
    ## If you want to retain the nan values, add "skipna=False"
    ds_weighted=(ds * weights).groupby("time.season").sum(dim="time",skipna=False)
    return ds_weighted

def season_mean_multiple(ds, calendar="standard"):
    # Get all the years by groupby
    ds_year=ds.groupby("time.year")
    # Get the first and last year
    years = list(ds_year.groups)
    first_year=min(years)
    last_year =max(years)
    # Do the seasonal means for every year
    for i in range(first_year,last_year+1):
        temp=season_mean(ds_year[i])
        if i==first_year:
            ds_temp=temp
        else:
            ds_temp=xr.concat([ds_temp,temp],dim="year")
    # Add a coordinate to the new dataset        
    ds_temp.coords["year"] = np.arange(first_year,last_year+1,1)
    return ds_temp

## 1.Read the model list

In [2]:
#---------------------- single------------------------------------------
with open('/zhoulab_rit/lzhuo/data/CMIP6/hist/v/single.txt') as f:
    single_list = [line.rstrip() for line in f]
f.close()
#---------------------- multiple------------------------------------------
with open('/zhoulab_rit/lzhuo/data/CMIP6/hist/v/multiple.txt') as f:
    multiple_list = [line.rstrip() for line in f]
f.close()

In [3]:
single_list

['ACCESS-CM2',
 'ACCESS-ESM1-5',
 'CanESM5',
 'CESM2-WACCM',
 'GFDL-ESM4',
 'TaiESM1']

In [4]:
multiple_list

['AWI-CM-1-1-MR',
 'CMCC-CM2-SR5',
 'CMCC-ESM2',
 'EC-Earth3',
 'EC-Earth3-Veg',
 'EC-Earth3-Veg-LR',
 'EC-Earth3-CC',
 'MIROC6',
 'MRI-ESM2-0']

## 2.Read model output with single file; note that after you process v, you need to modify the command below and run it again to process u

In [20]:
#------------------------- historical-------------------------------------------------
# path   = r'/zhoulab_rit/lzhuo/data/CMIP6/hist/v/'
path   = r'/zhoulab_rit/lzhuo/data/CMIP6/hist/u/'
length=len(single_list)
dss_list=[]
for i in range(0,length):
#     dss=glob.glob(os.path.join(path, "va_Amon_"+single_list[i]+"_historical*.nc"))
    dss=glob.glob(os.path.join(path, "ua_Amon_"+single_list[i]+"_historical*.nc"))
    dss_list.append(dss)

In [22]:
plevel=np.array([100000.,  92500.,  85000.,  70000.,  60000.,  50000.,  40000.,  30000.,
        25000.,  20000.,  15000.,  10000.,   7000.,   5000.,   3000.,   2000.,
         1000.,    500.,    100.])
plevel

array([100000.,  92500.,  85000.,  70000.,  60000.,  50000.,  40000.,
        30000.,  25000.,  20000.,  15000.,  10000.,   7000.,   5000.,
         3000.,   2000.,   1000.,    500.,    100.])

### process v

In [25]:
#--------------------------- historical --------------------------------------------------
length=len(dss_list)
for i in range(length):
    temp=dss_list[i]
    ds=xr.open_dataset(temp[0])
    #select data from 1985 to 2014
    va=ds.va.sel(time=slice('1985-01','2014-12'))
    #assign the same time coordinate
    va['time']=pd.date_range('1985-01-01','2014-12-01',freq='MS')
    #calculate the seasonal mean
    va_season=season_mean_multiple(va)
    #interpolate the data into same latitude, longitude and plev to make the concat possible
    new_lon = np.linspace(0, 359, 360)
    new_lat = np.linspace(-90,90,181)
    va_season_int=va_season.interp(lat=new_lat,lon=new_lon,plev=plevel)
    if i==0:
        va_single=va_season_int
    else:
        va_single=xr.concat([va_single,va_season_int],dim='model')
#assign the coordinate for the model
va_single['model']=np.arange(0,va_single.model.size)

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'va' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


### process u

In [9]:
#-------------------------- historical ----------------------------------------------
length=len(dss_list)
for i in range(length):
    temp=dss_list[i]
    ds=xr.open_dataset(temp[0])
    #select data from 1985 to 2014
    ua=ds.ua.sel(time=slice('1985-01','2014-12'))
    #assign the same time coordinate
    ua['time']=pd.date_range('1985-01-01','2014-12-01',freq='MS')
    #calculate the seasonal mean
    ua_season=season_mean_multiple(ua)
    #interpolate the data into same latitude, longitude and plev to make the concat possible
    new_lon = np.linspace(0, 359, 360)
    new_lat = np.linspace(-90,90,181)
    ua_season_int=ua_season.interp(lat=new_lat,lon=new_lon,plev=plevel)
    if i==0:
        ua_single=ua_season_int
    else:
        ua_single=xr.concat([ua_single,ua_season_int],dim='model')
#assign the coordinate for the model
ua_single['model']=np.arange(0,ua_single.model.size)

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'ua' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


## 3.Read model output with multiple files; same to the command above, rerun this command again after processing one variable

In [26]:
#----------------------------- historical simulation-------------------------------------
# path   = r'/zhoulab_rit/lzhuo/data/CMIP6/hist/v/'
path   = r'/zhoulab_rit/lzhuo/data/CMIP6/hist/u/'
length=len(multiple_list)
dss_list=[]
for i in range(0,length):
#     dss=glob.glob(os.path.join(path, "va_Amon_"+multiple_list[i]+"_historical*.nc"))
    dss=glob.glob(os.path.join(path, "ua_Amon_"+multiple_list[i]+"_historical*.nc"))
    dss_list.append(dss)

### process v

In [34]:
#-------------------------------- historical -----------------------------------------
length=len(dss_list)
for i in range(length):
    temp_list=dss_list[i]
    file_num=len(temp_list)
    for j in range(file_num):
        file=xr.open_dataset(temp_list[j])
        va=file.va
        if j==0:
            v=va
        else:
            v=xr.concat([v,va],dim='time')

    #select data from 1985 to 2014
    va=v.sel(time=slice('1985-01','2014-12'))
    #assign the same time coordinate
    va['time']=pd.date_range('1985-01-01','2014-12-01',freq='MS')
    #calculate the seasonal mean
    va_season=season_mean_multiple(va)
    #interpolate the data into same latitude and longitude to make the concat possible
    new_lon = np.linspace(0, 359, 360)
    new_lat = np.linspace(-90,90,181)
    va_season_int=va_season.interp(lat=new_lat,lon=new_lon,plev=plevel)
    if i==0:
        va_mul=va_season_int
    else:
        va_mul=xr.concat([va_mul,va_season_int],dim='model')

va_mul['model']=np.arange(6,6+va_mul.model.size)

### process u

In [13]:
length=len(dss_list)
for i in range(length):
    temp_list=dss_list[i]
    file_num=len(temp_list)
    for j in range(file_num):
        file=xr.open_dataset(temp_list[j])
        ua=file.ua
        if j==0:
            u=ua
        else:
            u=xr.concat([u,ua],dim='time')

    #select data from 1985 to 2014
    ua=u.sel(time=slice('1985-01','2014-12'))
    #assign the same time coordinate
    ua['time']=pd.date_range('1985-01-01','2014-12-01',freq='MS')
    #calculate the seasonal mean
    ua_season=season_mean_multiple(ua)
    #interpolate the data into same latitude and longitude to make the concat possible
    new_lon = np.linspace(0, 359, 360)
    new_lat = np.linspace(-90,90,181)
    ua_season_int=ua_season.interp(lat=new_lat,lon=new_lon,plev=plevel)
    if i==0:
        ua_mul=ua_season_int
    else:
        ua_mul=xr.concat([ua_mul,ua_season_int],dim='model')

ua_mul['model']=np.arange(6,6+ua_mul.model.size)

## 4.Concact the data with multiple files and single file

In [35]:
va_CMIP6=xr.concat([va_single,va_mul],dim='model')
va_CMIP6

<xarray.DataArray (model: 15, year: 30, season: 4, plev: 19, lat: 181, lon: 360)>
array([[[[[[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           ...,
           [-9.16839736e-01, -8.71764644e-01, -8.27505409e-01, ...,
            -1.02592118e+00, -9.96430530e-01,             nan],
           [-9.45123090e-01, -8.85260623e-01, -8.25148323e-01, ...,
            -1.12229974e+00, -1.06279340e+00,             nan],
           [-9.49407657e-01, -8.64318645e-01, -7.79177012e-01, ...,
            -1.20173951e+00, -1.11782718e+00,             nan]],

          [[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
...
           [-6.48406749e+00, -6.47281055e+00, -6.45993424e+00, ...,
            -6.50726377e+00, -6.50119115e+00,             nan],
           [-6.64599703e+00, -6.62804546e+00, -6.60829302e+00, ...,
            -6.68825529e+00, -6.67597387e+00,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan]],

          [[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [ 2.36541516e+00,  2.03172348e+00,  1.69733782e+00, ...,
             3.36079075e+00,  3.03000189e+00,             nan],
           [ 2.47641991e+00,  2.13683367e+00,  1.79632426e+00, ...,
             3.48788724e+00,  3.15199162e+00,             nan],
           ...,
           [-7.05551972e+00, -6.91639537e+00, -6.77533844e+00, ...,
            -7.46017755e+00, -7.32729700e+00,             nan],
           [-7.07935300e+00, -6.92574970e+00, -6.77013242e+00, ...,
            -7.52682229e+00, -7.37977792e+00,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan]]]]]])
Coordinates:
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * year     (year) int64 2071 2072 2073 2074 2075 ... 2096 2097 2098 2099 2100
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * plev     (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * model    (model) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

In [14]:
ua_CMIP6=xr.concat([ua_single,ua_mul],dim='model')
ua_CMIP6

<xarray.DataArray (model: 15, year: 30, season: 4, plev: 19, lat: 181, lon: 360)>
array([[[[[[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           ...,
           [-4.46222834e+00, -4.49843151e+00, -4.53235457e+00, ...,
            -4.43387553e+00, -4.44458090e+00,             nan],
           [-4.83009569e+00, -4.84311141e+00, -4.85602639e+00, ...,
            -4.78324554e+00, -4.79963639e+00,             nan],
           [-4.94458658e+00, -4.95967019e+00, -4.97431808e+00, ...,
            -4.88909324e+00, -4.90913963e+00,             nan]],

          [[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
...
           [ 2.44350818e+00,  2.33696562e+00,  2.23042216e+00, ...,
             2.76285018e+00,  2.65647509e+00,             nan],
           [ 5.21978449e-01,  4.09608871e-01,  2.97419636e-01, ...,
             8.59923193e-01,  7.47175491e-01,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan]],

          [[            nan,             nan,             nan, ...,
                        nan,             nan,             nan],
           [-1.72426576e+01, -1.72824422e+01, -1.73163681e+01, ...,
            -1.70858296e+01, -1.71439135e+01,             nan],
           [-1.57315625e+01, -1.57753187e+01, -1.58129988e+01, ...,
            -1.55615930e+01, -1.56242329e+01,             nan],
           ...,
           [-5.20725031e+00, -5.32872117e+00, -5.44830431e+00, ...,
            -4.83107745e+00, -4.95826287e+00,             nan],
           [-7.37800197e+00, -7.49941336e+00, -7.61842112e+00, ...,
            -6.99871584e+00, -7.12744809e+00,             nan],
           [            nan,             nan,             nan, ...,
                        nan,             nan,             nan]]]]]])
Coordinates:
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * year     (year) int64 2071 2072 2073 2074 2075 ... 2096 2097 2098 2099 2100
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * plev     (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * model    (model) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

## 5.Calculate the annual and seasonal mean data and write them as the dataset is very large 

In [15]:
ua_annual=ua_CMIP6.mean(dim='season')
ua_towrite=ua_annual.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_annual.nc')
ua_towrite=ua_annual.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_annual_fillna.nc')

ua_JJA=ua_CMIP6.sel(season='JJA')
ua_towrite=ua_JJA.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_JJA.nc')
ua_towrite=ua_JJA.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_JJA_fillna.nc')

ua_DJF=ua_CMIP6.sel(season='DJF')
ua_towrite=ua_DJF.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_DJF.nc')
ua_towrite=ua_DJF.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_DJF_fillna.nc')

In [36]:
va_annual=va_CMIP6.mean(dim='season')
va_towrite=va_annual.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_annual.nc')
va_towrite=va_annual.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_annual_fillna.nc')

va_JJA=va_CMIP6.sel(season='JJA')
va_towrite=va_JJA.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_JJA.nc')
va_towrite=va_JJA.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_JJA_fillna.nc')

va_DJF=va_CMIP6.sel(season='DJF')
va_towrite=va_DJF.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_DJF.nc')
va_towrite=va_DJF.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_DJF_fillna.nc')

## 6.Calculate the CMIP6 ensemble mean and write the data

In [37]:
#----------------------------------- historical ---------------------------------------------------
va_annual_MM=va_CMIP6.mean(dim=['season','model'])
va_towrite=va_annual_MM.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_annual_mmm.nc')
va_towrite=va_annual_MM.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_annual_mmm_fillna.nc')

va_MM=va_CMIP6.sel(season='JJA').mean(dim='model')
va_towrite=va_MM.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_JJA_mmm.nc')
va_towrite=va_MM.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_JJA_mmm_fillna.nc')

va_MM=va_CMIP6.sel(season='DJF').mean(dim='model')
va_towrite=va_MM.rename('va')
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_DJF_mmm.nc')
va_towrite=va_MM.rename('va').fillna(0.)
va_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/v/va_DJF_mmm_fillna.nc')

In [ ]:
ua_annual_MM=ua_CMIP6.mean(dim=['season','model'])
ua_towrite=ua_annual_MM.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_annual_mmm.nc')
ua_towrite=ua_annual_MM.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_annual_mmm_fillna.nc')

ua_MM=ua_CMIP6.sel(season='JJA').mean(dim='model')
ua_towrite=ua_MM.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_JJA_mmm.nc')
ua_towrite=ua_MM.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_JJA_mmm_fillna.nc')

ua_MM=ua_CMIP6.sel(season='DJF').mean(dim='model')
ua_towrite=ua_MM.rename('ua')
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_DJF_mmm.nc')
ua_towrite=ua_MM.rename('ua').fillna(0.)
ua_towrite.to_netcdf('/network/rit/lab/zhoulab_rit/lzhuo/data/CMIP6/hist/u/ua_DJF_mmm_fillna.nc')